<h1>Проект по SQL

**Описание проекта:**

Коронавирус застал мир врасплох, изменив привычный порядок вещей. В свободное время жители городов больше не выходят на улицу, не посещают кафе и торговые центры. Зато стало больше времени для книг. Это заметили стартаперы — и бросились создавать приложения для тех, кто любит читать.
Наша компания решила быть на волне и купила крупный сервис для чтения книг по подписке. Ваша первая задача как аналитика — проанализировать базу данных.
В ней — информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. Эти данные помогут сформулировать ценностное предложение для нового продукта.

**Описание данных:**

Таблица *books*
<br>Содержит данные о книгах:
- book_id — идентификатор книги;
- author_id — идентификатор автора;
- title — название книги;
- num_pages — количество страниц;
- publication_date — дата публикации книги;
- publisher_id — идентификатор издателя.

Таблица *authors*
<br>Содержит данные об авторах:
- author_id — идентификатор автора;
- author — имя автора.

Таблица *publishers*
<br>Содержит данные об издательствах:
- publisher_id — идентификатор издательства;
- publisher — название издательства;

Таблица *ratings*
<br>Содержит данные о пользовательских оценках книг:
- rating_id — идентификатор оценки;
- book_id — идентификатор книги;
- username — имя пользователя, оставившего оценку;
- rating — оценка книги.

Таблица *reviews*
<br>Содержит данные о пользовательских обзорах:
- review_id — идентификатор обзора;
- book_id — идентификатор книги;
- username — имя автора обзора;
- text — текст обзора.

**Шаги выполнения проекта:**
1. Исследуем таблицы — выведем первые строки;
2. Посчитаем, сколько книг вышло после 1 января 2000 года;
3. Для каждой книги посчитаем кол-во обзоров и среднюю оценку;
4. Определим издательство, которое выпустило наибольшее число книг толще 50 страниц — так мы исключим из анализа брошюры;
5. Определим автора с самой высокой средней оценкой книг — учитывать будем только книги с 50 и более оценками;
6. Посчитаем среднее кол-во обзоров от пользователей, которые поставили больше 48 оценок.
7. Опишим выводы по каждой из решённых задач.

**1. Исследуем таблицы**

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import text, create_engine
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

In [2]:
for i in ['books', 'authors', 'publishers', 'ratings', 'reviews']:
    query = '''SELECT * FROM {};'''.format(i) 
    request = pd.io.sql.read_sql(query, con = engine)
    print ('Таблица:', i)
    display(request.head())
    print()
    print(request.info())
    print()
    print()

Таблица: books


,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   book_id           1000 non-null   int64 
 1   author_id         1000 non-null   int64 
 2   title             1000 non-null   object
 3   num_pages         1000 non-null   int64 
 4   publication_date  1000 non-null   object
 5   publisher_id      1000 non-null   int64 
dtypes: int64(4), object(2)
memory usage: 47.0+ KB
None


Таблица: authors


,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 636 entries, 0 to 635
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   author_id  636 non-null    int64 
 1   author     636 non-null    object
dtypes: int64(1), object(1)
memory usage: 10.1+ KB
None


Таблица: publishers


...


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 340 entries, 0 to 339
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   publisher_id  340 non-null    int64 
 1   publisher     340 non-null    object
dtypes: int64(1), object(1)
memory usage: 5.4+ KB
None


Таблица: ratings


,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6456 entries, 0 to 6455
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   rating_id  6456 non-null   int64 
 1   book_id    6456 non-null   int64 
 2   username   6456 non-null   object
 3   rating     6456 non-null   int64 
dtypes: int64(3), object(1)
memory usage: 201.9+ KB
None


Таблица: reviews


,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2793 entries, 0 to 2792
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review_id  2793 non-null   int64 
 1   book_id    2793 non-null   int64 
 2   username   2793 non-null   object
 3   text       2793 non-null   object
dtypes: int64(2), object(2)
memory usage: 87.4+ KB
None




С данными на первый взгляд все хорошо, пропусков нет, с описанием все сходится, значит можно идти дальше.

**2. Сколько книг вышло после 1 января 2000 года**

In [3]:
query = '''
SELECT COUNT(*) FROM books WHERE publication_date >= '2000-01-01'
'''

request = pd.io.sql.read_sql(query, con = engine)
request

,count
0,821


Видим, что после 1 января 2020 года вышло 821 книг

**3. Количество обзоров и средняя оценка**

In [4]:
query = '''
SELECT books.title, 
COUNT(DISTINCT reviews.review_id) AS reviews_count,
AVG(ratings.rating) AS average_rating
FROM books
LEFT JOIN reviews ON books.book_id = reviews.book_id
LEFT JOIN ratings ON books.book_id = ratings.book_id
GROUP BY books.book_id;
'''

con=engine.connect()
request = pd.io.sql.read_sql(query, con = engine)
request

,title,reviews_count,average_rating
0,'Salem's Lot,2,3.666667
1,1 000 Places to See Before You Die,1,2.500000
2,13 Little Blue Envelopes (Little Blue Envelope...,3,4.666667
3,1491: New Revelations of the Americas Before C...,2,4.500000
4,1776,4,4.000000
...,...,...,...
995,Wyrd Sisters (Discworld #6; Witches #2),3,3.666667
996,Xenocide (Ender's Saga #3),3,3.400000
997,Year of Wonders,4,3.200000
998,You Suck (A Love Story #2),2,4.500000


**4. Издательство, которое выпустило наибольшее число книг**

Ищем среди книг толще 50 страниц — так мы исключим из анализа брошюры

In [5]:
query = '''
SELECT publishers.publisher AS publisher, 
COUNT(books.book_id) AS books_count
FROM publishers 
INNER JOIN books ON books.publisher_id = publishers.publisher_id 
WHERE num_pages > 50
GROUP BY publishers.publisher 
ORDER BY books_count DESC 
LIMIT 1;
'''

con=engine.connect()
request = pd.io.sql.read_sql(query, con = engine)
request

,publisher,books_count
0,Penguin Books,42


Издательство, которое выпустило наибольшее число книг — Penguin Books имеет 42 книги

**5. Автор с самой высокой средней оценкой книг**

Учитываем только книги с 50 и более оценками

In [6]:
query = '''
SELECT author, AVG(rating)
FROM authors
JOIN books ON authors.author_id = books.author_id
JOIN ratings AS rating ON books.book_id = rating.book_id
WHERE books.book_id IN 
(
SELECT book_id
FROM ratings 
GROUP BY book_id
HAVING COUNT(rating_id) >= 50
ORDER BY COUNT(rating_id) DESC
)
GROUP BY author
ORDER BY avg DESC
LIMIT 1
'''
con=engine.connect()
request = pd.io.sql.read_sql(query, con = engine)
request

,author,avg
0,J.K. Rowling/Mary GrandPré,4.287097


Автор с самой высокой средней оценкой книг - J.K. Rowling/Mary GrandPré имеет 4.29/5

**6. Среднее количество обзоров от пользователей**

Считаем среднее количество обзоров только от тех пользователей, которые поставили больше 48 оценок

In [7]:
query = '''
SELECT AVG(count)
FROM
(
SELECT COUNT(text)
FROM reviews FULL OUTER JOIN 
(
SELECT username, COUNT(rating_id) as count
FROM ratings
GROUP BY username
)
AS count_rating
ON count_rating.username = reviews.username
WHERE count > 48
GROUP BY reviews.username
)
as count
'''
con=engine.connect()
request = pd.io.sql.read_sql(query, con = engine)
request

,avg
0,24.0


Среднее количество обзоров только от тех пользователей, которые поставили больше 48 оценок = 24 балла

**7. Вывод:**

- Ознакомились с данными: все корректно, пропусков нет, с описанием все сошлось
- После 1 января 2020 года вышло 821 книг
- Для каждой книги посчитали кол-во обзоров и среднюю оценку
- Издательство, которое выпустило наибольшее число книг — Penguin Books имеет 42 книги
- Автор с самой высокой средней оценкой книг - J.K. Rowling/Mary GrandPré имеет 4.29/5
- Среднее количество обзоров только от тех пользователей, которые поставили больше 48 оценок = 24 балла